In [1]:
from sqlalchemy import create_engine, Column, String, Integer, Float, exc
from sqlalchemy.orm import Session, declarative_base

import requests

from selectorlib import Extractor
import requests
import json
from time import sleep
import random

In [2]:
# Create an Extractor by reading from the YAML file
def scrape(url, file):
    e = Extractor.from_yaml_file(file)


    user_agent_list = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36',
    'Opera/9.80 (Windows NT 6.1; WOW64) Presto/2.12.388 Version/12.18',
    'Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; .NET CLR 1.1.4322)',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.159 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
    ]
    url = 'https://httpbin.org/headers'
    for i in range(1,4):
    #Pick a random user agent
        user_agent = random.choice(user_agent_list)
    #Set the headers 
    
    headers = {
        'authority': 'www.amazon.com',
        'pragma': 'no-cache',
        'cache-control': 'no-cache',
        'dnt': '1',
        'upgrade-insecure-requests': '1',
        'user-agent':  user_agent,
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'sec-fetch-site': 'none',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-dest': 'document',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
    }
    # Download the page using requests
    r = requests.get(url, headers=headers)
    # Simple check to check if page was blocked (Usually 503)
    if r.status_code > 500:
        return None

    # Pass the HTML of the page and create
    return e.extract(r.text)

In [3]:
engine = create_engine("sqlite+pysqlite:///books.sqlite")
Base = declarative_base()
Base.metadata.create_all(engine)
session = Session(engine)

In [10]:
class Book(Base):
    __tablename__ = "Books"

    # New York Times API
    isbn = Column(String, primary_key=True, nullable=False)
    title = Column(String, nullable=False)
    author = Column(String, nullable=False)
    score = Column(Integer, nullable=False)
    publisher = Column(String, nullable=False)
    book_image = Column(String, nullable=True)
    amazon_product_url = Column(String, nullable=True)

    # TODO
    # Google books api
    # descriptions
    # genre

    amazon_rating = Column(Float, nullable=True)
    amazon_review_count = Column(Integer, nullable=True)

    goodreads_rating = Column(Float, nullable=True)
    goodreads_rating_count = Column(Integer, nullable=True)
    goodreads_review_count = Column(Integer, nullable=True)

    def __repr__(self) -> str:
        return f"Book(isbn={self.isbn!r}, title={self.title!r}, author={self.author!r}, score={self.score!r})"


In [65]:
print(len(test))
test = session.query(Book.isbn, Book.amazon_product_url).filter(Book.amazon_rating == None).all()
print(len(test))

2514
2422


In [67]:
for index, row in enumerate(test):

    print(row.isbn)
    amazon = scrape(row.amazon_product_url, "definitions/amazon.yml")

    print(amazon)
    #print(row.amazon_rating)

    if amazon != None:
        rating = amazon["rating"]
        if rating != None:
            amazon_rating = rating.split()[0]
            session.query(Book).filter(row.isbn == Book.isbn).update({'amazon_rating' : amazon_rating})
        reviewCount = amazon["reviewCount"]
        if reviewCount != None:
            amazon_review_count = reviewCount.split()[0].replace(",", "")
            session.query(Book).filter(row.isbn == Book.isbn).update({'amazon_review_count' : amazon_review_count})

    sleep(10)
    if index % 10 == 0:
        session.commit()

9780062353658
{'rating': None, 'reviewCount': None}
9780593098967
{'rating': None, 'reviewCount': None}


KeyboardInterrupt: 